In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import pandas as pd
import numpy as np

import itertools

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import confusion_matrix, roc_curve, auc

import matplotlib.pyplot as plt
%matplotlib inline

### In this notebook:
- I want to grab the data from 2013 - 2017 regular season for RegSeasonStats.
- I want NCAA tourney stats from 2013 - 2017.
- Ultimately, I want to use Reg Season Stats and NCAA Stats to predict 2018 NCAA tourney.
- To make the model, I want the Reg Season Stats from 2013, 2014, 2015, 2016, 2017 to predict the 2017 NCAA tourney and tune my model.
- Is there a way I can make a test set for 2014-2017 tourney results?  Can't use data from seasons after the year of the tourney I'm trying to predict.  Maybe I could convert Each Reg Season Stat to just "Previous Season Stat / WinRate" or "Previous Seed in tourney, or made it to NCAA finals the year before."

### Bringing in my edited DF's

In [2]:
df_reg = pd.read_csv('./RegularSeasonDetailedResultsEdited.csv')   ## This was edited in another notebook.

In [3]:
df_NCAA = pd.read_csv('./NCAATourneyDetailedResults_Edited.csv') ## This was edited in another notebook.

In [4]:
df_reg.columns

Index(['DayNum', 'NumOT', 'OppAst', 'OppBlk', 'OppDR', 'OppFGA', 'OppFGA3',
       'OppFGM', 'OppFGM3', 'OppFTA', 'OppFTM', 'OppLoc', 'OppOR', 'OppPF',
       'OppScore', 'OppStl', 'OppTO', 'OppTeamID', 'Season', 'TeamAst',
       'TeamBlk', 'TeamDR', 'TeamFGA', 'TeamFGA3', 'TeamFGM', 'TeamFGM3',
       'TeamFTA', 'TeamFTM', 'TeamLoc', 'TeamOR', 'TeamPF', 'TeamScore',
       'TeamStl', 'TeamTO', 'TeamTeamID', 'Won'],
      dtype='object')

In [5]:
df_NCAA.columns

Index(['DayNum', 'NumOT', 'OppAst', 'OppBlk', 'OppDR', 'OppFGA', 'OppFGA3',
       'OppFGM', 'OppFGM3', 'OppFTA', 'OppFTM', 'OppLoc', 'OppOR', 'OppPF',
       'OppScore', 'OppStl', 'OppTO', 'OppTeamID', 'Season', 'TeamAst',
       'TeamBlk', 'TeamDR', 'TeamFGA', 'TeamFGA3', 'TeamFGM', 'TeamFGM3',
       'TeamFTA', 'TeamFTM', 'TeamLoc', 'TeamOR', 'TeamPF', 'TeamScore',
       'TeamStl', 'TeamTO', 'TeamTeamID', 'Won'],
      dtype='object')

In [6]:
df_NCAA.head(4)

,DayNum,NumOT,OppAst,OppBlk,OppDR,OppFGA,OppFGA3,OppFGM,OppFGM3,OppFTA,...,TeamFTA,TeamFTM,TeamLoc,TeamOR,TeamPF,TeamScore,TeamStl,TeamTO,TeamTeamID,Won
0,134,1,16,0,28,67,31,29,12,31,...,26,17,N,14,22,92,5,12,1421,1
1,134,1,17,3,30,69,29,32,11,26,...,31,14,N,17,22,84,5,15,1411,0
2,136,0,12,3,26,64,16,20,4,7,...,14,11,N,11,8,80,10,16,1112,1
3,136,0,22,7,36,66,23,31,7,14,...,7,7,N,8,15,51,10,17,1436,0


In [7]:
df_reg.head(4)

,DayNum,NumOT,OppAst,OppBlk,OppDR,OppFGA,OppFGA3,OppFGM,OppFGM3,OppFTA,...,TeamFTA,TeamFTM,TeamLoc,TeamOR,TeamPF,TeamScore,TeamStl,TeamTO,TeamTeamID,Won
0,10,0,8,2,22,53,10,22,2,22,...,18,11,N,14,22,68,7,23,1104,1
1,10,0,13,1,24,58,14,27,3,18,...,22,16,N,10,20,62,9,18,1328,0
2,10,0,7,6,25,67,24,24,6,20,...,19,10,N,15,18,70,4,13,1272,1
3,10,0,16,4,28,62,20,26,8,19,...,20,9,N,20,16,63,8,12,1393,0


### Let's make the function for grabbing our stats

- Note: Not every team hear will have a stat for a given season, not every team makes it to D1.

In [8]:
def StatGrabber(input_df, input_year_list = [2013,2014,2015,2016,2017]):
    
    year_list = input_year_list
    
    loop_df = pd.DataFrame()
    for year in year_list:
        loop_df = pd.concat([df_reg.loc[df_reg['Season'] == int(year),:],loop_df], axis = 0)
    
    return loop_df

In [9]:
# df_reg.query(" Season == '2014' ").groupby('TeamTeamID').mean().rename(columns = {'Won': "2014_WinRate"}).columns
df_reg.query(" Season == '2014' ").groupby('TeamTeamID').mean().columns

Index(['DayNum', 'NumOT', 'OppAst', 'OppBlk', 'OppDR', 'OppFGA', 'OppFGA3',
       'OppFGM', 'OppFGM3', 'OppFTA', 'OppFTM', 'OppOR', 'OppPF', 'OppScore',
       'OppStl', 'OppTO', 'OppTeamID', 'Season', 'TeamAst', 'TeamBlk',
       'TeamDR', 'TeamFGA', 'TeamFGA3', 'TeamFGM', 'TeamFGM3', 'TeamFTA',
       'TeamFTM', 'TeamOR', 'TeamPF', 'TeamScore', 'TeamStl', 'TeamTO', 'Won'],
      dtype='object')

In [10]:
def StatGrabber(input_df, input_year_list, record_type = False):
    
    """ input_df :  Parent dataset, either RegularSeasonDetailedResultsEdited.csv or NCAATourneyDetailedResults_Edited.csv
        input_year_list : form of a list, really just something like "[2014]", only optimized to work one year at a time.
        record_type : Specify 'Reg' or 'NCAA' to add the str to each column label, help keep track of what stat belongs to which team.
    """
    
    year_list = input_year_list
    
    if record_type:
        record_type = "_" + str(record_type)
    else:
        record_type = ""
        
    loop_df = pd.DataFrame()
    for year in year_list:
        #We only want the stats labeled "Team", we only want a table in the form TeamTeamID, TeamStat1, TeamStat2... etc.
        loop_cols = [col for col in input_df.columns if "Team" in col and "Opp" not in col]
        loop_cols.append('Won')
        
        stat_df = input_df.loc[input_df['Season'] == int(year),loop_cols].copy()
        group_stat_df = stat_df.copy().groupby('TeamTeamID').mean().rename(columns = {'Won': "WinRate"})

        rename_dict = {}
        for col in group_stat_df.columns:
            rename_dict[col] = str(year) + str(record_type) + "_" + str(col).replace("Team","")
        
        group_stat_df.rename(columns = rename_dict, inplace = True)
        
#         loop_df = pd.concat([df_reg.loc[df_reg['Season'] == int(year),:],loop_df], axis = 0)
    
#     return loop_df
    return group_stat_df

In [11]:
StatGrabber(df_reg, input_year_list=[2017], record_type = "Reg").head()

,2017_Reg_Ast,2017_Reg_Blk,2017_Reg_DR,2017_Reg_FGA,2017_Reg_FGA3,2017_Reg_FGM,2017_Reg_FGM3,2017_Reg_FTA,2017_Reg_FTM,2017_Reg_OR,2017_Reg_PF,2017_Reg_Score,2017_Reg_Stl,2017_Reg_TO,2017_Reg_WinRate
TeamTeamID,,,,,,,,,,,,,,,
1101,13.600000,2.760000,22.880000,53.320000,19.080000,24.440000,7.080000,18.560000,11.920000,7.560000,22.160000,67.880000,7.000000,14.480000,0.360000
1102,14.483871,1.806452,25.258065,56.612903,21.870968,24.225806,7.935484,21.161290,15.161290,9.451613,17.903226,71.548387,5.903226,12.483871,0.322581
1103,14.515152,3.121212,24.575758,56.303030,27.303030,26.272727,10.090909,20.696970,14.030303,9.818182,18.363636,76.666667,5.424242,11.636364,0.757576
1104,11.424242,4.212121,26.878788,56.060606,20.787879,23.757576,6.606061,22.393939,14.545455,12.303030,19.727273,68.666667,6.000000,13.878788,0.575758
1105,11.379310,1.793103,21.862069,53.965517,15.758621,21.724138,4.310345,21.172414,14.103448,10.310345,18.137931,61.862069,3.551724,13.241379,0.068966


In [12]:
# This is cool, but it might be better for the model to consider all 2014-2017 tourneys with each row only having the team previous year's record.

In [13]:
NCAA_2017_stats = StatGrabber(df_NCAA, input_year_list=[2017], record_type = "NCAA")
Reg_2017_stats = StatGrabber(df_reg, input_year_list=[2017], record_type = "Reg")

In [14]:
target_df = df_NCAA.loc[df_NCAA['Season'] == 2017, ['TeamTeamID', 'OppTeamID', 'Won']].reset_index(drop = True)
target_df.head()

,TeamTeamID,OppTeamID,Won
0,1243,1448,1
1,1448,1243,0
2,1291,1309,1
3,1309,1291,0
4,1413,1300,1


In [15]:
merge1 = pd.merge(left = target_df, right = Reg_2017_stats, how = "inner", left_on = "TeamTeamID", right_index= True)
merge2 = pd.merge(left = merge1, right = Reg_2017_stats, how = 'inner', left_on = "OppTeamID", right_index = True, suffixes= {'_Team', '_Opp'})

merge3 = pd.merge(left = merge2, right = NCAA_2017_stats, how = 'inner', left_on = 'TeamTeamID', right_index = True)
merge4 = pd.merge(left = merge3, right = NCAA_2017_stats, how = 'inner', left_on = 'OppTeamID', right_index = True, suffixes= {'_Team', '_Opp'})

In [16]:
merge1.head()

,TeamTeamID,OppTeamID,Won,2017_Reg_Ast,2017_Reg_Blk,2017_Reg_DR,2017_Reg_FGA,2017_Reg_FGA3,2017_Reg_FGM,2017_Reg_FGM3,2017_Reg_FTA,2017_Reg_FTM,2017_Reg_OR,2017_Reg_PF,2017_Reg_Score,2017_Reg_Stl,2017_Reg_TO,2017_Reg_WinRate
0,1243,1448,1,14.969697,3.787879,23.242424,54.000000,19.666667,24.757576,7.090909,21.939394,15.121212,9.515152,17.727273,71.727273,7.818182,13.060606,0.606061
45,1243,1153,0,14.969697,3.787879,23.242424,54.000000,19.666667,24.757576,7.090909,21.939394,15.121212,9.515152,17.727273,71.727273,7.818182,13.060606,0.606061
1,1448,1243,0,15.562500,3.750000,26.062500,59.437500,20.937500,28.062500,8.093750,23.875000,18.437500,10.687500,20.093750,82.656250,5.468750,11.812500,0.593750
2,1291,1309,1,11.147059,4.000000,22.617647,55.088235,20.294118,24.441176,7.235294,18.088235,12.205882,7.647059,18.000000,68.323529,6.205882,13.147059,0.558824
31,1291,1437,0,11.147059,4.000000,22.617647,55.088235,20.294118,24.441176,7.235294,18.088235,12.205882,7.647059,18.000000,68.323529,6.205882,13.147059,0.558824


In [17]:
merge2.head()

,TeamTeamID,OppTeamID,Won,2017_Reg_Ast_Opp,2017_Reg_Blk_Opp,2017_Reg_DR_Opp,2017_Reg_FGA_Opp,2017_Reg_FGA3_Opp,2017_Reg_FGM_Opp,2017_Reg_FGM3_Opp,...,2017_Reg_FGM_Team,2017_Reg_FGM3_Team,2017_Reg_FTA_Team,2017_Reg_FTM_Team,2017_Reg_OR_Team,2017_Reg_PF_Team,2017_Reg_Score_Team,2017_Reg_Stl_Team,2017_Reg_TO_Team,2017_Reg_WinRate_Team
0,1243,1448,1,14.969697,3.787879,23.242424,54.000000,19.666667,24.757576,7.090909,...,28.062500,8.093750,23.875000,18.437500,10.687500,20.093750,82.656250,5.468750,11.812500,0.593750
45,1243,1153,0,14.969697,3.787879,23.242424,54.000000,19.666667,24.757576,7.090909,...,26.941176,7.411765,19.294118,13.176471,12.411765,15.970588,74.470588,7.470588,10.000000,0.852941
102,1417,1153,1,21.484848,5.393939,29.454545,64.969697,24.454545,33.727273,9.909091,...,26.941176,7.411765,19.294118,13.176471,12.411765,15.970588,74.470588,7.470588,10.000000,0.852941
1,1448,1243,0,15.562500,3.750000,26.062500,59.437500,20.937500,28.062500,8.093750,...,24.757576,7.090909,21.939394,15.121212,9.515152,17.727273,71.727273,7.818182,13.060606,0.606061
44,1153,1243,1,15.823529,5.088235,24.794118,59.294118,21.588235,26.941176,7.411765,...,24.757576,7.090909,21.939394,15.121212,9.515152,17.727273,71.727273,7.818182,13.060606,0.606061


In [18]:
merge4.shape

(134, 63)

In [19]:
merge4.head()

,TeamTeamID,OppTeamID,Won,2017_Reg_Ast_Opp,2017_Reg_Blk_Opp,2017_Reg_DR_Opp,2017_Reg_FGA_Opp,2017_Reg_FGA3_Opp,2017_Reg_FGM_Opp,2017_Reg_FGM3_Opp,...,2017_NCAA_FGM_Team,2017_NCAA_FGM3_Team,2017_NCAA_FTA_Team,2017_NCAA_FTM_Team,2017_NCAA_OR_Team,2017_NCAA_PF_Team,2017_NCAA_Score_Team,2017_NCAA_Stl_Team,2017_NCAA_TO_Team,2017_NCAA_WinRate_Team
0,1243,1448,1,14.969697,3.787879,23.242424,54.000000,19.666667,24.757576,7.090909,...,25.0,9.00,32.0,29.0,7.0,25.0,88.00,8.00,11.0,0.00
45,1243,1153,0,14.969697,3.787879,23.242424,54.000000,19.666667,24.757576,7.090909,...,26.5,6.00,16.0,12.0,8.0,12.5,71.00,3.00,10.5,0.50
102,1417,1153,1,21.484848,5.393939,29.454545,64.969697,24.454545,33.727273,9.909091,...,26.5,6.00,16.0,12.0,8.0,12.5,71.00,3.00,10.5,0.50
115,1417,1246,0,21.484848,5.393939,29.454545,64.969697,24.454545,33.727273,9.909091,...,27.0,6.75,20.5,15.0,10.5,15.5,75.75,5.75,8.5,0.75
124,1314,1246,1,18.212121,3.212121,27.484848,65.909091,19.757576,30.848485,7.242424,...,27.0,6.75,20.5,15.0,10.5,15.5,75.75,5.75,8.5,0.75


In [20]:
list(merge4.columns)

['TeamTeamID',
 'OppTeamID',
 'Won',
 '2017_Reg_Ast_Opp',
 '2017_Reg_Blk_Opp',
 '2017_Reg_DR_Opp',
 '2017_Reg_FGA_Opp',
 '2017_Reg_FGA3_Opp',
 '2017_Reg_FGM_Opp',
 '2017_Reg_FGM3_Opp',
 '2017_Reg_FTA_Opp',
 '2017_Reg_FTM_Opp',
 '2017_Reg_OR_Opp',
 '2017_Reg_PF_Opp',
 '2017_Reg_Score_Opp',
 '2017_Reg_Stl_Opp',
 '2017_Reg_TO_Opp',
 '2017_Reg_WinRate_Opp',
 '2017_Reg_Ast_Team',
 '2017_Reg_Blk_Team',
 '2017_Reg_DR_Team',
 '2017_Reg_FGA_Team',
 '2017_Reg_FGA3_Team',
 '2017_Reg_FGM_Team',
 '2017_Reg_FGM3_Team',
 '2017_Reg_FTA_Team',
 '2017_Reg_FTM_Team',
 '2017_Reg_OR_Team',
 '2017_Reg_PF_Team',
 '2017_Reg_Score_Team',
 '2017_Reg_Stl_Team',
 '2017_Reg_TO_Team',
 '2017_Reg_WinRate_Team',
 '2017_NCAA_Ast_Opp',
 '2017_NCAA_Blk_Opp',
 '2017_NCAA_DR_Opp',
 '2017_NCAA_FGA_Opp',
 '2017_NCAA_FGA3_Opp',
 '2017_NCAA_FGM_Opp',
 '2017_NCAA_FGM3_Opp',
 '2017_NCAA_FTA_Opp',
 '2017_NCAA_FTM_Opp',
 '2017_NCAA_OR_Opp',
 '2017_NCAA_PF_Opp',
 '2017_NCAA_Score_Opp',
 '2017_NCAA_Stl_Opp',
 '2017_NCAA_TO_Opp',
 '

In [21]:
'2017_Reg_OppAst-Team'.replace('-Opp', '').replace('2017', 'Opp_2017')

'Opp_2017_Reg_OppAst-Team'

In [22]:
## Looks loike we got the stats from the Regular season AND the NCAA Tournament. Pretty neat!

In [23]:
##Checking to see if it works in 2016

NCAA_2016_stats = StatGrabber(df_NCAA, input_year_list=[2016], record_type = "NCAA")
Reg_2016_stats = StatGrabber(df_reg, input_year_list=[2016], record_type = "Reg")

target_df = df_NCAA.loc[df_NCAA['Season'] == 2016, ['TeamTeamID', 'OppTeamID', 'Won']].reset_index(drop = True)

merge1 = pd.merge(left = target_df, right = Reg_2016_stats, how = "inner", left_on = "TeamTeamID", right_index= True)
merge2 = pd.merge(left = merge1, right = Reg_2016_stats, how = 'inner', left_on = "OppTeamID", right_index = True, suffixes= {'_Team', '_Opp'})

merge3 = pd.merge(left = merge2, right = NCAA_2016_stats, how = 'inner', left_on = 'TeamTeamID', right_index = True)
merge4 = pd.merge(left = merge3, right = NCAA_2016_stats, how = 'inner', left_on = 'OppTeamID', right_index = True, suffixes= {'_Team', '_Opp'})

In [24]:
merge4.shape

(134, 63)

In [25]:
merge4.head()

,TeamTeamID,OppTeamID,Won,2016_Reg_Ast_Opp,2016_Reg_Blk_Opp,2016_Reg_DR_Opp,2016_Reg_FGA_Opp,2016_Reg_FGA3_Opp,2016_Reg_FGM_Opp,2016_Reg_FGM3_Opp,...,2016_NCAA_FGM_Team,2016_NCAA_FGM3_Team,2016_NCAA_FTA_Team,2016_NCAA_FTM_Team,2016_NCAA_OR_Team,2016_NCAA_PF_Team,2016_NCAA_Score_Team,2016_NCAA_Stl_Team,2016_NCAA_TO_Team,2016_NCAA_WinRate_Team
0,1195,1192,1,12.896552,4.034483,27.965517,60.068966,14.655172,28.206897,5.068966,...,23.000000,8.000000,18.000000,11.000000,9.000000,26.000000,65.000000,9.000000,9.0,0.000000
29,1195,1314,0,12.896552,4.034483,27.965517,60.068966,14.655172,28.206897,5.068966,...,31.333333,6.833333,20.166667,16.166667,15.166667,18.166667,85.666667,5.833333,10.0,0.833333
113,1231,1314,0,15.968750,3.968750,25.343750,58.250000,23.812500,29.187500,9.875000,...,31.333333,6.833333,20.166667,16.166667,15.166667,18.166667,85.666667,5.833333,10.0,0.833333
85,1344,1314,0,15.909091,3.606061,24.909091,59.818182,22.090909,25.242424,7.090909,...,31.333333,6.833333,20.166667,16.166667,15.166667,18.166667,85.666667,5.833333,10.0,0.833333
125,1323,1314,0,13.468750,3.906250,25.156250,57.625000,19.906250,27.156250,7.343750,...,31.333333,6.833333,20.166667,16.166667,15.166667,18.166667,85.666667,5.833333,10.0,0.833333


In [26]:
merge4.columns

Index(['TeamTeamID', 'OppTeamID', 'Won', '2016_Reg_Ast_Opp',
       '2016_Reg_Blk_Opp', '2016_Reg_DR_Opp', '2016_Reg_FGA_Opp',
       '2016_Reg_FGA3_Opp', '2016_Reg_FGM_Opp', '2016_Reg_FGM3_Opp',
       '2016_Reg_FTA_Opp', '2016_Reg_FTM_Opp', '2016_Reg_OR_Opp',
       '2016_Reg_PF_Opp', '2016_Reg_Score_Opp', '2016_Reg_Stl_Opp',
       '2016_Reg_TO_Opp', '2016_Reg_WinRate_Opp', '2016_Reg_Ast_Team',
       '2016_Reg_Blk_Team', '2016_Reg_DR_Team', '2016_Reg_FGA_Team',
       '2016_Reg_FGA3_Team', '2016_Reg_FGM_Team', '2016_Reg_FGM3_Team',
       '2016_Reg_FTA_Team', '2016_Reg_FTM_Team', '2016_Reg_OR_Team',
       '2016_Reg_PF_Team', '2016_Reg_Score_Team', '2016_Reg_Stl_Team',
       '2016_Reg_TO_Team', '2016_Reg_WinRate_Team', '2016_NCAA_Ast_Opp',
       '2016_NCAA_Blk_Opp', '2016_NCAA_DR_Opp', '2016_NCAA_FGA_Opp',
       '2016_NCAA_FGA3_Opp', '2016_NCAA_FGM_Opp', '2016_NCAA_FGM3_Opp',
       '2016_NCAA_FTA_Opp', '2016_NCAA_FTM_Opp', '2016_NCAA_OR_Opp',
       '2016_NCAA_PF_Opp', '2016_NC

In [27]:
mask = [col for col in merge4.columns if "Rate" in col]
mask.append('TeamTeamID')
mask.append('OppTeamID')
mask.append('Won')
merge4.loc[:, mask].head(6)  ## Note that viewing WinRates for just a single year, NCAA Win Rate is kind of data leakage for who Won.

,2016_Reg_WinRate_Opp,2016_Reg_WinRate_Team,2016_NCAA_WinRate_Opp,2016_NCAA_WinRate_Team,TeamTeamID,OppTeamID,Won
0,0.551724,0.548387,0.500000,0.000000,1195,1192,1
29,0.551724,0.823529,0.500000,0.833333,1195,1314,0
113,0.781250,0.823529,0.666667,0.833333,1231,1314,0
85,0.696970,0.823529,0.500000,0.833333,1344,1314,0
125,0.656250,0.823529,0.750000,0.833333,1323,1314,0
129,0.593750,0.823529,0.800000,0.833333,1393,1314,0


In [182]:
## Lets try to make my function:

def ModelMerge(input_df_reg, input_df_NCAA, input_year):

    NCAA_stats = StatGrabber(input_df_NCAA, input_year_list=input_year, record_type = "NCAA")
    Reg_stats = StatGrabber(input_df_reg, input_year_list=input_year, record_type = "Reg")

    # Probably have to change this target for the final submission of ALL possible NCAA matchups
    target_df = input_df_NCAA.loc[input_df_NCAA['Season'] == input_year, ['TeamTeamID', 'OppTeamID', 'Won']].reset_index(drop = True)

    merge1 = pd.merge(left = target_df, right = Reg_stats, how = "inner", left_on = "TeamTeamID", right_index= True)
    merge2 = pd.merge(left = merge1, right = Reg_stats, how = 'inner', left_on = "OppTeamID", right_index = True, suffixes= {'_Team', '_Opp'})

    merge3 = pd.merge(left = merge2, right = NCAA_stats, how = 'inner', left_on = 'TeamTeamID', right_index = True)
    merge4 = pd.merge(left = merge3, right = NCAA_stats, how = 'inner', left_on = 'OppTeamID', right_index = True, suffixes= {'_Team', '_Opp'})
    
    return merge4

def StatsMerge(input_df_reg, input_df_NCAA, input_year, target_year):

    NCAA_stats = StatGrabber(input_df_NCAA, input_year_list=input_year, record_type = "NCAA")
    Reg_stats = StatGrabber(input_df_reg, input_year_list=input_year, record_type = "Reg")

    target_df = input_df_NCAA.loc[input_df_NCAA['Season'] == target_year, ['TeamTeamID', 'OppTeamID']].reset_index(drop = True)

    merge1 = pd.merge(left = target_df, right = Reg_stats, how = "inner", left_on = "TeamTeamID", right_index= True)
    merge2 = pd.merge(left = merge1, right = Reg_stats, how = 'inner', left_on = "OppTeamID", right_index = True, suffixes= {'_Team', '_Opp'})

    merge3 = pd.merge(left = merge2, right = NCAA_stats, how = 'inner', left_on = 'TeamTeamID', right_index = True)
    merge4 = pd.merge(left = merge3, right = NCAA_stats, how = 'inner', left_on = 'OppTeamID', right_index = True, suffixes= {'_Team', '_Opp'})
    
    return merge4

In [183]:
ModelMerge(df_reg, df_NCAA, [2013]).head()

,TeamTeamID,OppTeamID,Won,2013_Reg_Ast_Opp,2013_Reg_Blk_Opp,2013_Reg_DR_Opp,2013_Reg_FGA_Opp,2013_Reg_FGA3_Opp,2013_Reg_FGM_Opp,2013_Reg_FGM3_Opp,2013_Reg_FTA_Opp,2013_Reg_FTM_Opp,2013_Reg_OR_Opp,2013_Reg_PF_Opp,2013_Reg_Score_Opp,2013_Reg_Stl_Opp,2013_Reg_TO_Opp,2013_Reg_WinRate_Opp,2013_Reg_Ast_Team,2013_Reg_Blk_Team,2013_Reg_DR_Team,2013_Reg_FGA_Team,2013_Reg_FGA3_Team,2013_Reg_FGM_Team,2013_Reg_FGM3_Team,2013_Reg_FTA_Team,2013_Reg_FTM_Team,2013_Reg_OR_Team,2013_Reg_PF_Team,2013_Reg_Score_Team,2013_Reg_Stl_Team,2013_Reg_TO_Team,2013_Reg_WinRate_Team,2013_NCAA_Ast_Opp,2013_NCAA_Blk_Opp,2013_NCAA_DR_Opp,2013_NCAA_FGA_Opp,2013_NCAA_FGA3_Opp,2013_NCAA_FGM_Opp,2013_NCAA_FGM3_Opp,2013_NCAA_FTA_Opp,2013_NCAA_FTM_Opp,2013_NCAA_OR_Opp,2013_NCAA_PF_Opp,2013_NCAA_Score_Opp,2013_NCAA_Stl_Opp,2013_NCAA_TO_Opp,2013_NCAA_WinRate_Opp,2013_NCAA_Ast_Team,2013_NCAA_Blk_Team,2013_NCAA_DR_Team,2013_NCAA_FGA_Team,2013_NCAA_FGA3_Team,2013_NCAA_FGM_Team,2013_NCAA_FGM3_Team,2013_NCAA_FTA_Team,2013_NCAA_FTM_Team,2013_NCAA_OR_Team,2013_NCAA_PF_Team,2013_NCAA_Score_Team,2013_NCAA_Stl_Team,2013_NCAA_TO_Team,2013_NCAA_WinRate_Team
0,1299,1251,1,10.911765,4.529412,24.441176,51.911765,19.205882,20.558824,5.794118,22.411765,14.705882,10.647059,19.411765,61.617647,6.823529,15.058824,0.529412,11.625,2.906250,24.812500,53.843750,18.468750,22.687500,6.781250,20.750000,15.218750,10.062500,18.906250,67.375000,5.125000,13.312500,0.375000,8.500000,2.000000,21.000000,46.500000,17.000000,22.000000,7.000000,14.500000,9.500000,5.500000,18.5,60.500000,7.000000,17.500000,0.500000,15.000000,3.0,24.000000,54.000000,23.000000,23.000000,10.000000,18.000000,16.000000,6.000000,17.000000,72.0,4.000000,10.000000,0.0
21,1299,1257,0,10.911765,4.529412,24.441176,51.911765,19.205882,20.558824,5.794118,22.411765,14.705882,10.647059,19.411765,61.617647,6.823529,15.058824,0.529412,15.000,4.235294,23.882353,58.058824,17.617647,25.852941,5.823529,22.529412,16.029412,13.617647,17.735294,73.558824,10.735294,12.676471,0.852941,8.500000,2.000000,21.000000,46.500000,17.000000,22.000000,7.000000,14.500000,9.500000,5.500000,18.5,60.500000,7.000000,17.500000,0.500000,12.333333,4.0,21.333333,54.333333,15.333333,28.166667,5.333333,25.666667,17.833333,11.333333,20.666667,79.5,10.833333,10.666667,1.0
75,1161,1257,0,12.838710,1.806452,25.516129,56.290323,15.354839,24.935484,5.129032,24.354839,17.258065,14.451613,17.064516,72.258065,4.645161,10.709677,0.741935,15.000,4.235294,23.882353,58.058824,17.617647,25.852941,5.823529,22.529412,16.029412,13.617647,17.735294,73.558824,10.735294,12.676471,0.852941,7.000000,3.500000,23.500000,45.500000,13.000000,22.000000,6.000000,27.000000,20.000000,9.500000,18.0,70.000000,0.500000,17.000000,0.500000,12.333333,4.0,21.333333,54.333333,15.333333,28.166667,5.333333,25.666667,17.833333,11.333333,20.666667,79.5,10.833333,10.666667,1.0
133,1276,1257,0,14.125000,2.843750,24.312500,57.562500,19.562500,27.656250,7.343750,16.500000,11.718750,10.593750,13.062500,74.375000,6.031250,9.218750,0.781250,15.000,4.235294,23.882353,58.058824,17.617647,25.852941,5.823529,22.529412,16.029412,13.617647,17.735294,73.558824,10.735294,12.676471,0.852941,15.666667,2.333333,23.833333,58.666667,20.666667,28.333333,8.166667,15.833333,10.500000,10.666667,11.0,75.333333,6.833333,10.666667,0.833333,12.333333,4.0,21.333333,54.333333,15.333333,28.166667,5.333333,25.666667,17.833333,11.333333,20.666667,79.5,10.833333,10.666667,1.0
117,1332,1257,0,13.000000,4.000000,25.176471,56.970588,15.588235,25.470588,5.029412,22.205882,15.764706,12.235294,17.382353,71.735294,8.500000,14.911765,0.764706,15.000,4.235294,23.882353,58.058824,17.617647,25.852941,5.823529,22.529412,16.029412,13.617647,17.735294,73.558824,10.735294,12.676471,0.852941,14.000000,3.333333,27.333333,57.666667,15.666667,26.000000,7.000000,18.000000,11.333333,11.666667,20.0,70.333333,8.333333,16.000000,0.666667,12.333333,4.0,21.333333,54.333333,15.333333,28.166667,5.333333,25.666667,17.833333,11.333333,20.666667,79.5,10.833333,10.666667,1.

In [185]:
StatsMerge(df_reg, df_NCAA, [2014], 2014).head()  # Looks like this will work for each year!

,TeamTeamID,OppTeamID,2014_Reg_Ast_Opp,2014_Reg_Blk_Opp,2014_Reg_DR_Opp,2014_Reg_FGA_Opp,2014_Reg_FGA3_Opp,2014_Reg_FGM_Opp,2014_Reg_FGM3_Opp,2014_Reg_FTA_Opp,2014_Reg_FTM_Opp,2014_Reg_OR_Opp,2014_Reg_PF_Opp,2014_Reg_Score_Opp,2014_Reg_Stl_Opp,2014_Reg_TO_Opp,2014_Reg_WinRate_Opp,2014_Reg_Ast_Team,2014_Reg_Blk_Team,2014_Reg_DR_Team,2014_Reg_FGA_Team,2014_Reg_FGA3_Team,2014_Reg_FGM_Team,2014_Reg_FGM3_Team,2014_Reg_FTA_Team,2014_Reg_FTM_Team,2014_Reg_OR_Team,2014_Reg_PF_Team,2014_Reg_Score_Team,2014_Reg_Stl_Team,2014_Reg_TO_Team,2014_Reg_WinRate_Team,2014_NCAA_Ast_Opp,2014_NCAA_Blk_Opp,2014_NCAA_DR_Opp,2014_NCAA_FGA_Opp,2014_NCAA_FGA3_Opp,2014_NCAA_FGM_Opp,2014_NCAA_FGM3_Opp,2014_NCAA_FTA_Opp,2014_NCAA_FTM_Opp,2014_NCAA_OR_Opp,2014_NCAA_PF_Opp,2014_NCAA_Score_Opp,2014_NCAA_Stl_Opp,2014_NCAA_TO_Opp,2014_NCAA_WinRate_Opp,2014_NCAA_Ast_Team,2014_NCAA_Blk_Team,2014_NCAA_DR_Team,2014_NCAA_FGA_Team,2014_NCAA_FGA3_Team,2014_NCAA_FGM_Team,2014_NCAA_FGM3_Team,2014_NCAA_FTA_Team,2014_NCAA_FTM_Team,2014_NCAA_OR_Team,2014_NCAA_PF_Team,2014_NCAA_Score_Team,2014_NCAA_Stl_Team,2014_NCAA_TO_Team,2014_NCAA_WinRate_Team
0,1107,1291,11.062500,3.062500,24.218750,51.093750,13.468750,22.437500,4.906250,21.718750,16.250000,10.343750,17.250000,66.031250,5.531250,12.625000,0.562500,11.750000,3.062500,22.125,58.218750,24.906250,25.656250,8.875000,21.687500,16.062500,10.062500,18.750000,76.250000,7.156250,12.000000,0.500000,11.0,1.500000,28.500000,49.000000,11.000000,23.500000,3.0,16.500000,13.000000,6.000000,14.000000,63.000000,3.500000,12.0,0.50,13.0,1.0,17.0,60.0,37.0,22.0,12.0,12.0,8.0,6.0,16.0,64.0,3.0,4.0,0.0
13,1107,1196,11.062500,3.062500,24.218750,51.093750,13.468750,22.437500,4.906250,21.718750,16.250000,10.343750,17.250000,66.031250,5.531250,12.625000,0.562500,13.352941,2.852941,24.000,52.764706,18.617647,24.382353,6.852941,22.882353,15.117647,11.676471,16.205882,70.735294,7.205882,11.323529,0.941176,11.0,1.500000,28.500000,49.000000,11.000000,23.500000,3.0,16.500000,13.000000,6.000000,14.000000,63.000000,3.500000,12.0,0.50,11.0,4.2,23.8,52.4,15.6,23.2,4.4,18.6,13.6,10.6,12.0,64.4,5.8,9.2,0.8
128,1163,1196,12.588235,6.205882,25.294118,54.382353,18.382353,24.382353,7.117647,21.000000,15.970588,10.088235,18.264706,71.852941,6.970588,11.676471,0.764706,13.352941,2.852941,24.000,52.764706,18.617647,24.382353,6.852941,22.882353,15.117647,11.676471,16.205882,70.735294,7.205882,11.323529,0.941176,10.0,3.166667,24.833333,52.166667,19.333333,23.666667,7.5,19.166667,16.833333,7.333333,16.333333,71.666667,7.333333,10.0,1.00,11.0,4.2,23.8,52.4,15.6,23.2,4.4,18.6,13.6,10.6,12.0,64.4,5.8,9.2,0.8
121,1173,1196,13.060606,2.606061,23.666667,55.060606,17.393939,25.666667,6.575758,22.727273,15.484848,11.636364,20.060606,73.393939,5.818182,12.000000,0.696970,13.352941,2.852941,24.000,52.764706,18.617647,24.382353,6.852941,22.882353,15.117647,11.676471,16.205882,70.735294,7.205882,11.323529,0.941176,13.5,2.500000,23.500000,50.250000,17.500000,22.000000,6.5,17.250000,11.750000,7.500000,18.500000,62.250000,7.500000,11.5,0.75,11.0,4.2,23.8,52.4,15.6,23.2,4.4,18.6,13.6,10.6,12.0,64.4,5.8,9.2,0.8
77,1338,1196,15.617647,3.000000,23.941176,53.882353,14.441176,24.852941,5.147059,24.794118,17.617647,12.147059,17.500000,72.470588,7.000000,10.558824,0.735294,13.352941,2.852941,24.000,52.764706,18.617647,24.382353,6.852941,22.882353,15.117647,11.676471,16.205882,70.735294,7.205882,11.323529,0.941176,16.0,2.500000,22.500000,56.000000,15.500000,25.000000,4.0,9.000000,7.000000,9.500000,12.500000,61.000000,8.000000,7.0,0.50,11.0,4.2,23.8,52.4,15.6,23.2,4.4,18.6,13.6,10.6,12.0,64.4,5.8,9.2,0.8


In [31]:
def StatGrabber(input_df, input_year_list, record_type = False):
    
    """ input_df :  Parent dataset, either RegularSeasonDetailedResultsEdited.csv or NCAATourneyDetailedResults_Edited.csv
        input_year_list : form of a list, really just something like "[2014]", only optimized to work one year at a time.
        record_type : Specify 'Reg' or 'NCAA' to add the str to each column label, help keep track of what stat belongs to which team.
    """
    
    year_list = input_year_list
    
    if record_type:
        record_type = "_" + str(record_type)
    else:
        record_type = ""
        
    loop_df = pd.DataFrame()
    for year in year_list:
        #We only want the stats labeled "Team", we only want a table in the form TeamTeamID, TeamStat1, TeamStat2... etc.
        loop_cols = [col for col in input_df.columns if "Team" in col and "Opp" not in col]
        loop_cols.append('Won')
        
        stat_df = input_df.loc[input_df['Season'] == int(year),loop_cols].copy()
        group_stat_df = stat_df.copy().groupby('TeamTeamID').mean().rename(columns = {'Won': "WinRate"})

        rename_dict = {}
        for col in group_stat_df.columns:
            rename_dict[col] = str(year) + str(record_type) + "_" + str(col).replace("Team","")
        
        group_stat_df.rename(columns = rename_dict, inplace = True)
        
#         loop_df = pd.concat([loop_df, group_stat_df], axis = 1)
    
#     return loop_df
    return group_stat_df

In [186]:
target_year = 2016

allstats_2013 = StatsMerge(df_reg, df_NCAA, [2013], target_year)
allstats_2014 = StatsMerge(df_reg, df_NCAA, [2014], target_year)
allstats_2015 = StatsMerge(df_reg, df_NCAA, [2015], target_year)
allstats_2016 = StatsMerge(df_reg, df_NCAA, [2016], target_year)
allstats_2017 = StatsMerge(df_reg, df_NCAA, [2017], target_year)

target_df = df_NCAA.loc[df_NCAA['Season'] == 2016, ['TeamTeamID', 'OppTeamID', 'Won']].reset_index(drop = True)

In [187]:
allstats_2017.head()

,TeamTeamID,OppTeamID,2017_Reg_Ast_Opp,2017_Reg_Blk_Opp,2017_Reg_DR_Opp,2017_Reg_FGA_Opp,2017_Reg_FGA3_Opp,2017_Reg_FGM_Opp,2017_Reg_FGM3_Opp,2017_Reg_FTA_Opp,2017_Reg_FTM_Opp,2017_Reg_OR_Opp,2017_Reg_PF_Opp,2017_Reg_Score_Opp,2017_Reg_Stl_Opp,2017_Reg_TO_Opp,2017_Reg_WinRate_Opp,2017_Reg_Ast_Team,2017_Reg_Blk_Team,2017_Reg_DR_Team,2017_Reg_FGA_Team,2017_Reg_FGA3_Team,2017_Reg_FGM_Team,2017_Reg_FGM3_Team,2017_Reg_FTA_Team,2017_Reg_FTM_Team,2017_Reg_OR_Team,2017_Reg_PF_Team,2017_Reg_Score_Team,2017_Reg_Stl_Team,2017_Reg_TO_Team,2017_Reg_WinRate_Team,2017_NCAA_Ast_Opp,2017_NCAA_Blk_Opp,2017_NCAA_DR_Opp,2017_NCAA_FGA_Opp,2017_NCAA_FGA3_Opp,2017_NCAA_FGM_Opp,2017_NCAA_FGM3_Opp,2017_NCAA_FTA_Opp,2017_NCAA_FTM_Opp,2017_NCAA_OR_Opp,2017_NCAA_PF_Opp,2017_NCAA_Score_Opp,2017_NCAA_Stl_Opp,2017_NCAA_TO_Opp,2017_NCAA_WinRate_Opp,2017_NCAA_Ast_Team,2017_NCAA_Blk_Team,2017_NCAA_DR_Team,2017_NCAA_FGA_Team,2017_NCAA_FGA3_Team,2017_NCAA_FGM_Team,2017_NCAA_FGM3_Team,2017_NCAA_FTA_Team,2017_NCAA_FTM_Team,2017_NCAA_OR_Team,2017_NCAA_PF_Team,2017_NCAA_Score_Team,2017_NCAA_Stl_Team,2017_NCAA_TO_Team,2017_NCAA_WinRate_Team
29,1195,1314,14.000000,3.366667,25.633333,58.033333,18.466667,28.233333,6.266667,19.700000,13.700000,11.300000,18.933333,76.433333,5.266667,12.333333,0.766667,18.212121,3.212121,27.484848,65.909091,19.757576,30.848485,7.242424,21.787879,15.393939,15.757576,17.909091,84.333333,6.787879,11.878788,0.787879,18.0,1.0,19.0,70.0,28.0,29.0,8.0,20.0,14.0,7.0,23.0,80.0,6.0,11.0,0.0,17.166667,4.833333,29.500000,65.666667,21.0,28.5,6.166667,27.166667,18.5,15.500000,17.666667,81.666667,7.833333,11.333333,1.000000
85,1344,1314,14.187500,3.000000,24.562500,54.500000,19.531250,24.343750,7.187500,20.812500,14.375000,8.968750,17.125000,70.250000,6.718750,12.593750,0.625000,18.212121,3.212121,27.484848,65.909091,19.757576,30.848485,7.242424,21.787879,15.393939,15.757576,17.909091,84.333333,6.787879,11.878788,0.787879,17.0,2.0,20.0,51.0,20.0,25.0,9.0,17.0,12.0,7.0,21.0,71.0,3.0,9.0,0.0,17.166667,4.833333,29.500000,65.666667,21.0,28.5,6.166667,27.166667,18.5,15.500000,17.666667,81.666667,7.833333,11.333333,1.000000
125,1323,1314,16.000000,3.764706,24.441176,59.794118,24.411765,27.558824,9.411765,16.852941,13.470588,9.323529,14.852941,78.000000,7.176471,9.441176,0.735294,18.212121,3.212121,27.484848,65.909091,19.757576,30.848485,7.242424,21.787879,15.393939,15.757576,17.909091,84.333333,6.787879,11.878788,0.787879,11.0,3.0,23.5,53.0,19.5,21.5,7.0,19.0,15.5,8.5,15.0,65.5,5.0,10.0,0.5,17.166667,4.833333,29.500000,65.666667,21.0,28.5,6.166667,27.166667,18.5,15.500000,17.666667,81.666667,7.833333,11.333333,1.000000
132,1437,1314,14.294118,3.088235,24.794118,54.029412,23.823529,26.852941,8.823529,19.088235,15.147059,8.558824,14.205882,77.676471,7.235294,11.205882,0.911765,18.212121,3.212121,27.484848,65.909091,19.757576,30.848485,7.242424,21.787879,15.393939,15.757576,17.909091,84.333333,6.787879,11.878788,0.787879,9.5,2.0,24.0,55.5,16.5,26.0,5.5,16.0,11.5,12.5,16.5,69.0,6.5,10.5,0.5,17.166667,4.833333,29.500000,65.666667,21.0,28.5,6.166667,27.166667,18.5,15.500000,17.666667,81.666667,7.833333,11.333333,1.000000
30,1344,1425,14.187500,3.000000,24.562500,54.500000,19.531250,24.343750,7.187500,20.812500,14.375000,8.968750,17.125000,70.250000,6.718750,12.593750,0.625000,15.303030,5.424242,25.303030,59.909091,21.606061,27.181818,7.848485,22.303030,16.484848,10.787879,15.909091,78.696970,7.242424,11.151515,0.727273,17.0,2.0,20.0,51.0,20.0,25.0,9.0,17.0,12.0,7.0,21.0,71.0,3.0,9.0,0.0,15.000000,3.666667,20.666667,54.000000,23.0,26.0,8.000000,17.000000,13.0,10.333333,15.666667,73.000000,4.333333,10.000000,0.666667


In [188]:
target_df.head(2)

,TeamTeamID,OppTeamID,Won
0,1195,1192,1
1,1192,1195,0


In [189]:
#Remember, only want to pick the stats from teams playing in the 2017 NCAA Tournament
allstats_2013.shape, allstats_2014.shape, allstats_2015.shape,allstats_2016.shape, allstats_2017.shape, target_df.shape

((40, 62), (44, 62), (44, 62), (134, 62), (44, 62), (134, 3))

In [120]:
# Dont edit this one

loop_df = target_df.copy()

loop_df['Match'] = loop_df['TeamTeamID'].astype(str) + "_" + loop_df['OppTeamID'].astype(str)
loop_df.drop(['TeamTeamID', 'OppTeamID'], axis = 1, inplace = True)

df_list = [allstats_2013, allstats_2014, allstats_2015, allstats_2016, allstats_2017]

# for table in df_list:
    
loop_merge_df = allstats_2013.copy()
loop_merge_df['Match'] = loop_merge_df['TeamTeamID'].astype(str) + "_" + loop_merge_df['OppTeamID'].astype(str)
loop_merge_df.drop(['TeamTeamID', 'OppTeamID'], axis = 1, inplace = True)

merge1 = pd.merge(left = loop_df, right = loop_merge_df, how = 'inner', on = "Match")
# merge2 = pd.merge(left = merge1, right = loop_merge_df, how = 'inner', on = "Match")
    
#     loop_df = pd.merge(left = loop_df, right = merge2, how = "right", )
merge1.shape

(134, 62)

In [121]:
merge1.sample(5)  ## Looks like this is working, we're only grabbing the stats from teams that have played eachother in both Reg n NCAA

,Won,Match,2017_Reg_Ast_Opp,2017_Reg_Blk_Opp,2017_Reg_DR_Opp,2017_Reg_FGA_Opp,2017_Reg_FGA3_Opp,2017_Reg_FGM_Opp,2017_Reg_FGM3_Opp,2017_Reg_FTA_Opp,2017_Reg_FTM_Opp,2017_Reg_OR_Opp,2017_Reg_PF_Opp,2017_Reg_Score_Opp,2017_Reg_Stl_Opp,2017_Reg_TO_Opp,2017_Reg_WinRate_Opp,2017_Reg_Ast_Team,2017_Reg_Blk_Team,2017_Reg_DR_Team,2017_Reg_FGA_Team,2017_Reg_FGA3_Team,2017_Reg_FGM_Team,2017_Reg_FGM3_Team,2017_Reg_FTA_Team,2017_Reg_FTM_Team,2017_Reg_OR_Team,2017_Reg_PF_Team,2017_Reg_Score_Team,2017_Reg_Stl_Team,2017_Reg_TO_Team,2017_Reg_WinRate_Team,2017_NCAA_Ast_Opp,2017_NCAA_Blk_Opp,2017_NCAA_DR_Opp,2017_NCAA_FGA_Opp,2017_NCAA_FGA3_Opp,2017_NCAA_FGM_Opp,2017_NCAA_FGM3_Opp,2017_NCAA_FTA_Opp,2017_NCAA_FTM_Opp,2017_NCAA_OR_Opp,2017_NCAA_PF_Opp,2017_NCAA_Score_Opp,2017_NCAA_Stl_Opp,2017_NCAA_TO_Opp,2017_NCAA_WinRate_Opp,2017_NCAA_Ast_Team,2017_NCAA_Blk_Team,2017_NCAA_DR_Team,2017_NCAA_FGA_Team,2017_NCAA_FGA3_Team,2017_NCAA_FGM_Team,2017_NCAA_FGM3_Team,2017_NCAA_FTA_Team,2017_NCAA_FTM_Team,2017_NCAA_OR_Team,2017_NCAA_PF_Team,2017_NCAA_Score_Team,2017_NCAA_Stl_Team,2017_NCAA_TO_Team,2017_NCAA_WinRate_Team
93,0,2017_1455_1246,16.424242,3.787879,28.848485,58.939394,21.424242,27.787879,8.727273,24.000000,17.606061,11.909091,18.848485,81.909091,6.757576,11.121212,0.878788,15.529412,5.294118,27.588235,63.294118,19.970588,30.176471,7.058824,26.441176,18.529412,12.588235,19.294118,85.941176,6.058824,12.029412,0.852941,11.5,6.5,29.5,55.0,22.0,21.5,7.0,18.0,13.0,13.5,16.5,63.0,1.0,13.5,0.5,14.25,5.250000,25.750000,61.000000,19.250000,27.000000,6.750000,20.5,15.000000,10.50,15.500000,75.750000,5.750000,8.500000,0.750000
79,0,2017_1321_1211,15.264706,5.000000,25.441176,58.029412,20.794118,25.294118,7.088235,17.852941,13.411765,10.764706,19.029412,71.088235,5.029412,10.529412,0.676471,16.030303,4.454545,30.939394,58.636364,19.121212,30.363636,7.303030,22.424242,16.545455,9.363636,16.454545,84.575758,7.151515,11.303030,0.969697,11.0,2.5,26.5,59.5,19.0,26.5,7.0,15.5,10.5,10.0,22.0,70.5,9.0,12.5,0.5,11.00,6.000000,31.333333,57.666667,19.666667,24.500000,7.500000,25.0,15.333333,11.50,19.833333,71.833333,4.833333,12.500000,0.833333
32,1,2017_1438_1423,14.250000,4.062500,24.125000,53.656250,17.812500,24.906250,7.000000,13.812500,9.781250,8.812500,16.250000,66.593750,5.656250,9.531250,0.687500,15.656250,3.437500,23.125000,63.125000,26.406250,30.093750,9.500000,19.500000,13.437500,11.812500,19.875000,83.125000,6.500000,9.781250,0.843750,8.5,4.0,25.0,53.0,16.0,21.0,4.0,16.0,11.5,7.5,18.0,57.5,4.5,9.5,0.5,15.00,2.000000,26.000000,61.000000,26.000000,25.000000,10.000000,17.0,11.000000,14.00,20.000000,71.000000,4.000000,9.000000,0.000000
122,1,2017_1332_1242,16.333333,6.545455,25.636364,58.060606,22.727273,27.969697,8.606061,20.000000,14.121212,10.272727,16.303030,78.666667,6.545455,11.727273,0.848485,16.281250,4.687500,26.812500,60.718750,21.531250,29.562500,8.718750,22.343750,14.875000,12.031250,18.125000,82.718750,6.937500,13.093750,0.875000,11.2,4.4,26.8,57.6,21.4,27.4,8.4,19.8,14.2,11.4,14.0,77.4,6.4,12.0,0.8,15.75,3.000000,26.500000,62.500000,24.500000,31.250000,9.750000,18.5,14.750000,10.75,15.250000,87.000000,7.000000,9.250000,0.750000
43,0,2017_1308_1124,14.666667,3.500000,26.433333,56.566667,22.133333,26.166667,7.400000,24.966667,17.700000,12.466667,18.600000,77.433333,4.866667,13.733333,0.833333,15.483871,4.870968,24.935484,55.225806,18.709677,25.774194,6.612903,18.903226,13.451613,12.677419,17.677419,71.612903,5.129032,13.419355,0.774194,15.0,2.0,18.0,64.0,22.0,28.0,8.0,13.0,9.0,9.0,18.0,73.0,6.0,11.0,0.0,14.00,4.333333,23.666667,59.333333,13.333333,27.666667,4.666667,18.0,14.333333,13.00,17.000000,74.333333,6.000000,13.333333,0.666667


In [38]:
pd.options.display.max_rows = 40
pd.options.display.max_columns = 302

In [194]:
target_year = 2017

allstats_2013 = StatsMerge(df_reg, df_NCAA, [2013], target_year)
allstats_2014 = StatsMerge(df_reg, df_NCAA, [2014], target_year)
allstats_2015 = StatsMerge(df_reg, df_NCAA, [2015], target_year)
allstats_2016 = StatsMerge(df_reg, df_NCAA, [2016], target_year)
allstats_2017 = StatsMerge(df_reg, df_NCAA, [2017], target_year)

target_df = df_NCAA.loc[df_NCAA['Season'] == 2017, ['TeamTeamID', 'OppTeamID', 'Won']].reset_index(drop = True)

loop_df = target_df.copy()

loop_df['Match'] = loop_df['TeamTeamID'].astype(str) + "_" + loop_df['OppTeamID'].astype(str)
loop_df.drop(['TeamTeamID', 'OppTeamID'], axis = 1, inplace = True)

df_list = [
    allstats_2017, 
    allstats_2016, 
    allstats_2015, 
    allstats_2014, 
    allstats_2013]

for table in df_list:
    
    loop_merge_df = table.copy()
    loop_merge_df['Match'] = loop_merge_df['TeamTeamID'].astype(str) + "_" + loop_merge_df['OppTeamID'].astype(str)
    loop_merge_df.drop(['TeamTeamID', 'OppTeamID'], axis = 1, inplace = True)

    loop_df = pd.merge(left = loop_df, right = loop_merge_df, how = 'left', on = "Match")
# merge2 = pd.merge(left = merge1, right = loop_merge_df, how = 'inner', on = "Match")
    
#     loop_df = pd.merge(left = loop_df, right = merge2, how = "right", )

In [195]:
loop_df.shape

(134, 302)

In [196]:
target_df.head()

,TeamTeamID,OppTeamID,Won
0,1243,1448,1
1,1448,1243,0
2,1291,1309,1
3,1309,1291,0
4,1413,1300,1


In [197]:
loop_df.head(5)
#This df has all of the stats I want to use, but need to drop tourney results from the X set in model.
# also need to fill nan values somehow.

#  The final submission needs to be the predictions for every year. This df could work for 2018

,Won,Match,2017_Reg_Ast_Opp,2017_Reg_Blk_Opp,2017_Reg_DR_Opp,2017_Reg_FGA_Opp,2017_Reg_FGA3_Opp,2017_Reg_FGM_Opp,2017_Reg_FGM3_Opp,2017_Reg_FTA_Opp,2017_Reg_FTM_Opp,2017_Reg_OR_Opp,2017_Reg_PF_Opp,2017_Reg_Score_Opp,2017_Reg_Stl_Opp,2017_Reg_TO_Opp,2017_Reg_WinRate_Opp,2017_Reg_Ast_Team,2017_Reg_Blk_Team,2017_Reg_DR_Team,2017_Reg_FGA_Team,2017_Reg_FGA3_Team,2017_Reg_FGM_Team,2017_Reg_FGM3_Team,2017_Reg_FTA_Team,2017_Reg_FTM_Team,2017_Reg_OR_Team,2017_Reg_PF_Team,2017_Reg_Score_Team,2017_Reg_Stl_Team,2017_Reg_TO_Team,2017_Reg_WinRate_Team,2017_NCAA_Ast_Opp,2017_NCAA_Blk_Opp,2017_NCAA_DR_Opp,2017_NCAA_FGA_Opp,2017_NCAA_FGA3_Opp,2017_NCAA_FGM_Opp,2017_NCAA_FGM3_Opp,2017_NCAA_FTA_Opp,2017_NCAA_FTM_Opp,2017_NCAA_OR_Opp,2017_NCAA_PF_Opp,2017_NCAA_Score_Opp,2017_NCAA_Stl_Opp,2017_NCAA_TO_Opp,2017_NCAA_WinRate_Opp,2017_NCAA_Ast_Team,2017_NCAA_Blk_Team,2017_NCAA_DR_Team,2017_NCAA_FGA_Team,2017_NCAA_FGA3_Team,2017_NCAA_FGM_Team,2017_NCAA_FGM3_Team,2017_NCAA_FTA_Team,2017_NCAA_FTM_Team,2017_NCAA_OR_Team,2017_NCAA_PF_Team,2017_NCAA_Score_Team,2017_NCAA_Stl_Team,2017_NCAA_TO_Team,2017_NCAA_WinRate_Team,2016_Reg_Ast_Opp,2016_Reg_Blk_Opp,2016_Reg_DR_Opp,2016_Reg_FGA_Opp,2016_Reg_FGA3_Opp,2016_Reg_FGM_Opp,2016_Reg_FGM3_Opp,2016_Reg_FTA_Opp,2016_Reg_FTM_Opp,2016_Reg_OR_Opp,2016_Reg_PF_Opp,2016_Reg_Score_Opp,2016_Reg_Stl_Opp,2016_Reg_TO_Opp,2016_Reg_WinRate_Opp,2016_Reg_Ast_Team,2016_Reg_Blk_Team,2016_Reg_DR_Team,2016_Reg_FGA_Team,2016_Reg_FGA3_Team,2016_Reg_FGM_Team,2016_Reg_FGM3_Team,2016_Reg_FTA_Team,2016_Reg_FTM_Team,2016_Reg_OR_Team,2016_Reg_PF_Team,2016_Reg_Score_Team,2016_Reg_Stl_Team,2016_Reg_TO_Team,2016_Reg_WinRate_Team,2016_NCAA_Ast_Opp,2016_NCAA_Blk_Opp,2016_NCAA_DR_Opp,2016_NCAA_FGA_Opp,2016_NCAA_FGA3_Opp,2016_NCAA_FGM_Opp,2016_NCAA_FGM3_Opp,2016_NCAA_FTA_Opp,2016_NCAA_FTM_Opp,2016_NCAA_OR_Opp,2016_NCAA_PF_Opp,2016_NCAA_Score_Opp,2016_NCAA_Stl_Opp,2016_NCAA_TO_Opp,2016_NCAA_WinRate_Opp,2016_NCAA_Ast_Team,2016_NCAA_Blk_Team,2016_NCAA_DR_Team,2016_NCAA_FGA_Team,2016_NCAA_FGA3_Team,2016_NCAA_FGM_Team,2016_NCAA_FGM3_Team,2016_NCAA_FTA_Team,2016_NCAA_FTM_Team,2016_NCAA_OR_Team,2016_NCAA_PF_Team,2016_NCAA_Score_Team,2016_NCAA_Stl_Team,2016_NCAA_TO_Team,2016_NCAA_WinRate_Team,2015_Reg_Ast_Opp,2015_Reg_Blk_Opp,2015_Reg_DR_Opp,2015_Reg_FGA_Opp,2015_Reg_FGA3_Opp,2015_Reg_FGM_Opp,2015_Reg_FGM3_Opp,2015_Reg_FTA_Opp,2015_Reg_FTM_Opp,2015_Reg_OR_Opp,2015_Reg_PF_Opp,2015_Reg_Score_Opp,2015_Reg_Stl_Opp,2015_Reg_TO_Opp,2015_Reg_WinRate_Opp,2015_Reg_Ast_Team,2015_Reg_Blk_Team,2015_Reg_DR_Team,2015_Reg_FGA_Team,2015_Reg_FGA3_Team,2015_Reg_FGM_Team,2015_Reg_FGM3_Team,2015_Reg_FTA_Team,2015_Reg_FTM_Team,2015_Reg_OR_Team,2015_Reg_PF_Team,2015_Reg_Score_Team,2015_Reg_Stl_Team,2015_Reg_TO_Team,2015_Reg_WinRate_Team,2015_NCAA_Ast_Opp,2015_NCAA_Blk_Opp,2015_NCAA_DR_Opp,2015_NCAA_FGA_Opp,2015_NCAA_FGA3_Opp,2015_NCAA_FGM_Opp,2015_NCAA_FGM3_Opp,2015_NCAA_FTA_Opp,2015_NCAA_FTM_Opp,2015_NCAA_OR_Opp,2015_NCAA_PF_Opp,2015_NCAA_Score_Opp,2015_NCAA_Stl_Opp,2015_NCAA_TO_Opp,2015_NCAA_WinRate_Opp,2015_NCAA_Ast_Team,2015_NCAA_Blk_Team,2015_NCAA_DR_Team,2015_NCAA_FGA_Team,2015_NCAA_FGA3_Team,2015_NCAA_FGM_Team,2015_NCAA_FGM3_Team,2015_NCAA_FTA_Team,2015_NCAA_FTM_Team,2015_NCAA_OR_Team,2015_NCAA_PF_Team,2015_NCAA_Score_Team,2015_NCAA_Stl_Team,2015_NCAA_TO_Team,2015_NCAA_WinRate_Team,2014_Reg_Ast_Opp,2014_Reg_Blk_Opp,2014_Reg_DR_Opp,2014_Reg_FGA_Opp,2014_Reg_FGA3_Opp,2014_Reg_FGM_Opp,2014_Reg_FGM3_Opp,2014_Reg_FTA_Opp,2014_Reg_FTM_Opp,2014_Reg_OR_Opp,2014_Reg_PF_Opp,2014_Reg_Score_Opp,2014_Reg_Stl_Opp,2014_Reg_TO_Opp,2014_Reg_WinRate_Opp,2014_Reg_Ast_Team,2014_Reg_Blk_Team,2014_Reg_DR_Team,2014_Reg_FGA_Team,2014_Reg_FGA3_Team,2014_Reg_FGM_Team,2014_Reg_FGM3_Team,2014_Reg_FTA_Team,2014_Reg_FTM_Team,2014_Reg_OR_Team,2014_Reg_PF_Team,2014_Reg_Score_Team,2014_Reg_Stl_Team,2014_Reg_TO_Team,2014_Reg_WinRate_Team,2014_NCAA_Ast_Opp,2014_NCAA_Blk_Opp,2014_NCAA_DR_Opp,2014_NCAA_FGA_Opp,2014_NCAA_FGA3_Opp,2014_NCAA_FGM_Opp,2014_NCAA_FGM3_Opp,2014_NCAA_FTA_Opp,2014_NCAA_FTM_Opp,2014_

In [247]:
# Now trying to do predictions for just the 2017 winners, filling nans with 0 for teams that didnt have records for a given year.

mask = [col for col in loop_df.columns if "2017_NCAA" not in col]

train_df = loop_df.copy().fillna(value = 0.0)

X = train_df.copy().loc[:, mask].drop('Won', axis = 1).set_index('Match')
y = train_df['Won'].copy()

X_train, X_hold, y_train, y_hold = train_test_split(X,y, test_size = 0.25, random_state = 1)

ss = StandardScaler()
Xs_train = ss.fit_transform(X_train)
Xs_hold = ss.transform(X_hold)

model = LogisticRegressionCV(Cs = np.linspace(0.1,100,50), cv = 3)
# model = RandomForestClassifier()

model.fit(Xs_train, y_train)

print(model.score(Xs_train, y_train), model.score(Xs_hold, y_hold))

print(cross_val_score(model, X_train,y_train).mean())

0.97 0.5882352941176471
0.4893048128342246


In [199]:
loop_df.head()

,Won,Match,2017_Reg_Ast_Opp,2017_Reg_Blk_Opp,2017_Reg_DR_Opp,2017_Reg_FGA_Opp,2017_Reg_FGA3_Opp,2017_Reg_FGM_Opp,2017_Reg_FGM3_Opp,2017_Reg_FTA_Opp,2017_Reg_FTM_Opp,2017_Reg_OR_Opp,2017_Reg_PF_Opp,2017_Reg_Score_Opp,2017_Reg_Stl_Opp,2017_Reg_TO_Opp,2017_Reg_WinRate_Opp,2017_Reg_Ast_Team,2017_Reg_Blk_Team,2017_Reg_DR_Team,2017_Reg_FGA_Team,2017_Reg_FGA3_Team,2017_Reg_FGM_Team,2017_Reg_FGM3_Team,2017_Reg_FTA_Team,2017_Reg_FTM_Team,2017_Reg_OR_Team,2017_Reg_PF_Team,2017_Reg_Score_Team,2017_Reg_Stl_Team,2017_Reg_TO_Team,2017_Reg_WinRate_Team,2017_NCAA_Ast_Opp,2017_NCAA_Blk_Opp,2017_NCAA_DR_Opp,2017_NCAA_FGA_Opp,2017_NCAA_FGA3_Opp,2017_NCAA_FGM_Opp,2017_NCAA_FGM3_Opp,2017_NCAA_FTA_Opp,2017_NCAA_FTM_Opp,2017_NCAA_OR_Opp,2017_NCAA_PF_Opp,2017_NCAA_Score_Opp,2017_NCAA_Stl_Opp,2017_NCAA_TO_Opp,2017_NCAA_WinRate_Opp,2017_NCAA_Ast_Team,2017_NCAA_Blk_Team,2017_NCAA_DR_Team,2017_NCAA_FGA_Team,2017_NCAA_FGA3_Team,2017_NCAA_FGM_Team,2017_NCAA_FGM3_Team,2017_NCAA_FTA_Team,2017_NCAA_FTM_Team,2017_NCAA_OR_Team,2017_NCAA_PF_Team,2017_NCAA_Score_Team,2017_NCAA_Stl_Team,2017_NCAA_TO_Team,2017_NCAA_WinRate_Team,2016_Reg_Ast_Opp,2016_Reg_Blk_Opp,2016_Reg_DR_Opp,2016_Reg_FGA_Opp,2016_Reg_FGA3_Opp,2016_Reg_FGM_Opp,2016_Reg_FGM3_Opp,2016_Reg_FTA_Opp,2016_Reg_FTM_Opp,2016_Reg_OR_Opp,2016_Reg_PF_Opp,2016_Reg_Score_Opp,2016_Reg_Stl_Opp,2016_Reg_TO_Opp,2016_Reg_WinRate_Opp,2016_Reg_Ast_Team,2016_Reg_Blk_Team,2016_Reg_DR_Team,2016_Reg_FGA_Team,2016_Reg_FGA3_Team,2016_Reg_FGM_Team,2016_Reg_FGM3_Team,2016_Reg_FTA_Team,2016_Reg_FTM_Team,2016_Reg_OR_Team,2016_Reg_PF_Team,2016_Reg_Score_Team,2016_Reg_Stl_Team,2016_Reg_TO_Team,2016_Reg_WinRate_Team,2016_NCAA_Ast_Opp,2016_NCAA_Blk_Opp,2016_NCAA_DR_Opp,2016_NCAA_FGA_Opp,2016_NCAA_FGA3_Opp,2016_NCAA_FGM_Opp,2016_NCAA_FGM3_Opp,2016_NCAA_FTA_Opp,2016_NCAA_FTM_Opp,2016_NCAA_OR_Opp,2016_NCAA_PF_Opp,2016_NCAA_Score_Opp,2016_NCAA_Stl_Opp,2016_NCAA_TO_Opp,2016_NCAA_WinRate_Opp,2016_NCAA_Ast_Team,2016_NCAA_Blk_Team,2016_NCAA_DR_Team,2016_NCAA_FGA_Team,2016_NCAA_FGA3_Team,2016_NCAA_FGM_Team,2016_NCAA_FGM3_Team,2016_NCAA_FTA_Team,2016_NCAA_FTM_Team,2016_NCAA_OR_Team,2016_NCAA_PF_Team,2016_NCAA_Score_Team,2016_NCAA_Stl_Team,2016_NCAA_TO_Team,2016_NCAA_WinRate_Team,2015_Reg_Ast_Opp,2015_Reg_Blk_Opp,2015_Reg_DR_Opp,2015_Reg_FGA_Opp,2015_Reg_FGA3_Opp,2015_Reg_FGM_Opp,2015_Reg_FGM3_Opp,2015_Reg_FTA_Opp,2015_Reg_FTM_Opp,2015_Reg_OR_Opp,2015_Reg_PF_Opp,2015_Reg_Score_Opp,2015_Reg_Stl_Opp,2015_Reg_TO_Opp,2015_Reg_WinRate_Opp,2015_Reg_Ast_Team,2015_Reg_Blk_Team,2015_Reg_DR_Team,2015_Reg_FGA_Team,2015_Reg_FGA3_Team,2015_Reg_FGM_Team,2015_Reg_FGM3_Team,2015_Reg_FTA_Team,2015_Reg_FTM_Team,2015_Reg_OR_Team,2015_Reg_PF_Team,2015_Reg_Score_Team,2015_Reg_Stl_Team,2015_Reg_TO_Team,2015_Reg_WinRate_Team,2015_NCAA_Ast_Opp,2015_NCAA_Blk_Opp,2015_NCAA_DR_Opp,2015_NCAA_FGA_Opp,2015_NCAA_FGA3_Opp,2015_NCAA_FGM_Opp,2015_NCAA_FGM3_Opp,2015_NCAA_FTA_Opp,2015_NCAA_FTM_Opp,2015_NCAA_OR_Opp,2015_NCAA_PF_Opp,2015_NCAA_Score_Opp,2015_NCAA_Stl_Opp,2015_NCAA_TO_Opp,2015_NCAA_WinRate_Opp,2015_NCAA_Ast_Team,2015_NCAA_Blk_Team,2015_NCAA_DR_Team,2015_NCAA_FGA_Team,2015_NCAA_FGA3_Team,2015_NCAA_FGM_Team,2015_NCAA_FGM3_Team,2015_NCAA_FTA_Team,2015_NCAA_FTM_Team,2015_NCAA_OR_Team,2015_NCAA_PF_Team,2015_NCAA_Score_Team,2015_NCAA_Stl_Team,2015_NCAA_TO_Team,2015_NCAA_WinRate_Team,2014_Reg_Ast_Opp,2014_Reg_Blk_Opp,2014_Reg_DR_Opp,2014_Reg_FGA_Opp,2014_Reg_FGA3_Opp,2014_Reg_FGM_Opp,2014_Reg_FGM3_Opp,2014_Reg_FTA_Opp,2014_Reg_FTM_Opp,2014_Reg_OR_Opp,2014_Reg_PF_Opp,2014_Reg_Score_Opp,2014_Reg_Stl_Opp,2014_Reg_TO_Opp,2014_Reg_WinRate_Opp,2014_Reg_Ast_Team,2014_Reg_Blk_Team,2014_Reg_DR_Team,2014_Reg_FGA_Team,2014_Reg_FGA3_Team,2014_Reg_FGM_Team,2014_Reg_FGM3_Team,2014_Reg_FTA_Team,2014_Reg_FTM_Team,2014_Reg_OR_Team,2014_Reg_PF_Team,2014_Reg_Score_Team,2014_Reg_Stl_Team,2014_Reg_TO_Team,2014_Reg_WinRate_Team,2014_NCAA_Ast_Opp,2014_NCAA_Blk_Opp,2014_NCAA_DR_Opp,2014_NCAA_FGA_Opp,2014_NCAA_FGA3_Opp,2014_NCAA_FGM_Opp,2014_NCAA_FGM3_Opp,2014_NCAA_FTA_Opp,2014_NCAA_FTM_Opp,2014_

In [244]:
target_year = 2017

allstats_2013 = StatsMerge(df_reg, df_NCAA, [2013], target_year)
allstats_2014 = StatsMerge(df_reg, df_NCAA, [2014], target_year)
allstats_2015 = StatsMerge(df_reg, df_NCAA, [2015], target_year)
allstats_2016 = StatsMerge(df_reg, df_NCAA, [2016], target_year)
allstats_2017 = StatsMerge(df_reg, df_NCAA, [2017], target_year)

target_df = df_NCAA.loc[df_NCAA['Season'] == target_year, ['TeamTeamID', 'OppTeamID', 'Won']].reset_index(drop = True)

loop_df = target_df.copy()

loop_df['Match'] = loop_df['TeamTeamID'].astype(str) + "_" + loop_df['OppTeamID'].astype(str)
loop_df.drop(['TeamTeamID', 'OppTeamID'], axis = 1, inplace = True)

df_list = [
    allstats_2017, 
    allstats_2016, 
    allstats_2015, 
    allstats_2014, 
    allstats_2013
]

for table in df_list:
    
    loop_merge_df = table.copy()
    loop_merge_df['Match'] = loop_merge_df['TeamTeamID'].astype(str) + "_" + loop_merge_df['OppTeamID'].astype(str)
    loop_merge_df.drop(['TeamTeamID', 'OppTeamID'], axis = 1, inplace = True)

    loop_df = pd.merge(left = loop_df, right = loop_merge_df, how = 'left', on = "Match")
# merge2 = pd.merge(left = merge1, right = loop_merge_df, how = 'inner', on = "Match")
    
#     loop_df = pd.merge(left = loop_df, right = merge2, how = "right", )

In [246]:
mask2 = [col for col in loop_df.columns if str(target_year) + "_Reg" in col]
mask2.append('Won')
mask2.append('Match')

loop_df.loc[:,mask2].head()

,2017_Reg_Ast_Opp,2017_Reg_Blk_Opp,2017_Reg_DR_Opp,2017_Reg_FGA_Opp,2017_Reg_FGA3_Opp,2017_Reg_FGM_Opp,2017_Reg_FGM3_Opp,2017_Reg_FTA_Opp,2017_Reg_FTM_Opp,2017_Reg_OR_Opp,2017_Reg_PF_Opp,2017_Reg_Score_Opp,2017_Reg_Stl_Opp,2017_Reg_TO_Opp,2017_Reg_WinRate_Opp,2017_Reg_Ast_Team,2017_Reg_Blk_Team,2017_Reg_DR_Team,2017_Reg_FGA_Team,2017_Reg_FGA3_Team,2017_Reg_FGM_Team,2017_Reg_FGM3_Team,2017_Reg_FTA_Team,2017_Reg_FTM_Team,2017_Reg_OR_Team,2017_Reg_PF_Team,2017_Reg_Score_Team,2017_Reg_Stl_Team,2017_Reg_TO_Team,2017_Reg_WinRate_Team,Won,Match
0,14.969697,3.787879,23.242424,54.000000,19.666667,24.757576,7.090909,21.939394,15.121212,9.515152,17.727273,71.727273,7.818182,13.060606,0.606061,15.562500,3.750000,26.062500,59.437500,20.937500,28.062500,8.093750,23.875000,18.437500,10.687500,20.093750,82.656250,5.468750,11.812500,0.593750,1,1243_1448
1,15.562500,3.750000,26.062500,59.437500,20.937500,28.062500,8.093750,23.875000,18.437500,10.687500,20.093750,82.656250,5.468750,11.812500,0.593750,14.969697,3.787879,23.242424,54.000000,19.666667,24.757576,7.090909,21.939394,15.121212,9.515152,17.727273,71.727273,7.818182,13.060606,0.606061,0,1448_1243
2,11.147059,4.000000,22.617647,55.088235,20.294118,24.441176,7.235294,18.088235,12.205882,7.647059,18.000000,68.323529,6.205882,13.147059,0.558824,15.071429,3.000000,23.178571,54.000000,13.535714,24.928571,4.214286,23.500000,16.678571,11.928571,17.607143,70.750000,7.392857,16.571429,0.607143,1,1291_1309
3,15.071429,3.000000,23.178571,54.000000,13.535714,24.928571,4.214286,23.500000,16.678571,11.928571,17.607143,70.750000,7.392857,16.571429,0.607143,11.147059,4.000000,22.617647,55.088235,20.294118,24.441176,7.235294,18.088235,12.205882,7.647059,18.000000,68.323529,6.205882,13.147059,0.558824,0,1309_1291
4,12.500000,3.500000,25.218750,55.687500,17.468750,24.062500,6.125000,22.875000,15.125000,10.218750,19.687500,69.375000,5.562500,14.187500,0.625000,13.633333,3.266667,26.733333,57.466667,21.933333,26.100000,7.300000,19.966667,13.833333,11.033333,17.166667,73.333333,5.866667,12.233333,0.733333,1,1413_1300


In [233]:
train_df = loop_df.copy()

X = train_df.copy().loc[:, mask2].drop('Won', axis = 1).set_index('Match')
y = train_df['Won'].copy()

X_train, X_hold, y_train, y_hold = train_test_split(X,y, test_size = 0.25, random_state = 1)

ss = StandardScaler()
Xs_train = ss.fit_transform(X_train)
Xs_hold = ss.transform(X_hold)

model = LogisticRegressionCV(Cs = np.logspace(-3,3,7), cv = 3)
# model = RandomForestClassifier()

model.fit(Xs_train, y_train)

print(model.score(Xs_train, y_train), model.score(Xs_hold, y_hold))

print(cross_val_score(model, Xs_hold,y_hold).mean())

0.66 0.47058823529411764
0.5353535353535354


In [220]:
## Looks like just using the stats from the regular season isn't a good method for predicting the NCAA tournament

## I don think this is working the way i think it is, going to try rebuilding my traindf just using stat grabber

In [249]:
regstats2017 = StatGrabber(df_reg,[2017], 'Reg')

target_df = df_NCAA.loc[df_NCAA['Season'] == 2017, ['TeamTeamID', 'OppTeamID', 'Won']].reset_index(drop = True)

m1 = pd.merge(left = target_df, right = regstats2017, left_on = 'TeamTeamID', right_index = True)
m2 = pd.merge(left = m1, right = regstats2017, left_on = 'OppTeamID', right_index = True, suffixes={"_Team", "_Opp"})

In [255]:
m2.head()

,TeamTeamID,OppTeamID,Won,2017_Reg_Ast_Opp,2017_Reg_Blk_Opp,2017_Reg_DR_Opp,2017_Reg_FGA_Opp,2017_Reg_FGA3_Opp,2017_Reg_FGM_Opp,2017_Reg_FGM3_Opp,2017_Reg_FTA_Opp,2017_Reg_FTM_Opp,2017_Reg_OR_Opp,2017_Reg_PF_Opp,2017_Reg_Score_Opp,2017_Reg_Stl_Opp,2017_Reg_TO_Opp,2017_Reg_WinRate_Opp,2017_Reg_Ast_Team,2017_Reg_Blk_Team,2017_Reg_DR_Team,2017_Reg_FGA_Team,2017_Reg_FGA3_Team,2017_Reg_FGM_Team,2017_Reg_FGM3_Team,2017_Reg_FTA_Team,2017_Reg_FTM_Team,2017_Reg_OR_Team,2017_Reg_PF_Team,2017_Reg_Score_Team,2017_Reg_Stl_Team,2017_Reg_TO_Team,2017_Reg_WinRate_Team
0,1243,1448,1,14.969697,3.787879,23.242424,54.000000,19.666667,24.757576,7.090909,21.939394,15.121212,9.515152,17.727273,71.727273,7.818182,13.060606,0.606061,15.562500,3.750000,26.062500,59.437500,20.937500,28.062500,8.093750,23.875000,18.437500,10.687500,20.093750,82.656250,5.468750,11.812500,0.593750
45,1243,1153,0,14.969697,3.787879,23.242424,54.000000,19.666667,24.757576,7.090909,21.939394,15.121212,9.515152,17.727273,71.727273,7.818182,13.060606,0.606061,15.823529,5.088235,24.794118,59.294118,21.588235,26.941176,7.411765,19.294118,13.176471,12.411765,15.970588,74.470588,7.470588,10.000000,0.852941
102,1417,1153,1,21.484848,5.393939,29.454545,64.969697,24.454545,33.727273,9.909091,17.454545,13.000000,9.757576,16.454545,90.363636,6.121212,11.545455,0.878788,15.823529,5.088235,24.794118,59.294118,21.588235,26.941176,7.411765,19.294118,13.176471,12.411765,15.970588,74.470588,7.470588,10.000000,0.852941
1,1448,1243,0,15.562500,3.750000,26.062500,59.437500,20.937500,28.062500,8.093750,23.875000,18.437500,10.687500,20.093750,82.656250,5.468750,11.812500,0.593750,14.969697,3.787879,23.242424,54.000000,19.666667,24.757576,7.090909,21.939394,15.121212,9.515152,17.727273,71.727273,7.818182,13.060606,0.606061
44,1153,1243,1,15.823529,5.088235,24.794118,59.294118,21.588235,26.941176,7.411765,19.294118,13.176471,12.411765,15.970588,74.470588,7.470588,10.000000,0.852941,14.969697,3.787879,23.242424,54.000000,19.666667,24.757576,7.090909,21.939394,15.121212,9.515152,17.727273,71.727273,7.818182,13.060606,0.606061


In [261]:
# Model for 2017, using only 2017 reg season stats.
train_df = m2.copy()

train_df['Match'] = train_df['TeamTeamID'].astype(str) + "_" + train_df['OppTeamID'].astype(str)
train_df.drop(['TeamTeamID', 'OppTeamID'], axis = 1, inplace = True)

X = train_df.copy().drop('Won', axis = 1).set_index('Match')
y = train_df['Won'].copy()

X_train, X_hold, y_train, y_hold = train_test_split(X,y, test_size = 0.3, random_state = 1)

ss = StandardScaler()
Xs_train = ss.fit_transform(X_train)
Xs_hold = ss.transform(X_hold)

model = LogisticRegressionCV(Cs = np.logspace(-3,3,7), cv = 3)
# model = RandomForestClassifier()

model.fit(Xs_train, y_train)

print(model.score(Xs_train, y_train), model.score(Xs_hold, y_hold))

print(cross_val_score(model, Xs_hold,y_hold).mean())

0.5913978494623656 0.6829268292682927
0.6615384615384615


In [264]:
## Predicting 2017 off of just 2016 reg season, looks like this is working somewhat.
## Looks like it's better practice to just build stats one at a time, I was running in to problems trying to merge each stat by year, then
## trying to join on Match.

regstats2016 = StatGrabber(df_reg,[2016], 'Reg')

target_df = df_NCAA.loc[df_NCAA['Season'] == 2017, ['TeamTeamID', 'OppTeamID', 'Won']].reset_index(drop = True)

m1 = pd.merge(left = target_df, right = regstats2016, left_on = 'TeamTeamID', right_index = True)
m2 = pd.merge(left = m1, right = regstats2016, left_on = 'OppTeamID', right_index = True, suffixes={"_Team", "_Opp"})

train_df = m2.copy()

train_df['Match'] = train_df['TeamTeamID'].astype(str) + "_" + train_df['OppTeamID'].astype(str)
train_df.drop(['TeamTeamID', 'OppTeamID'], axis = 1, inplace = True)

X = train_df.copy().drop('Won', axis = 1).set_index('Match')
y = train_df['Won'].copy()

X_train, X_hold, y_train, y_hold = train_test_split(X,y, test_size = 0.25, random_state = 1)

ss = StandardScaler()
Xs_train = ss.fit_transform(X_train)
Xs_hold = ss.transform(X_hold)

model = LogisticRegressionCV(Cs = np.logspace(-3,3,7), cv = 3)
# model = RandomForestClassifier()

model.fit(Xs_train, y_train)

print(model.score(Xs_train, y_train), model.score(Xs_hold, y_hold))

print(cross_val_score(model, Xs_hold,y_hold).mean())

0.75 0.6764705882352942
0.7676767676767677


In [401]:
def SingleMerger(df1, df2):
    m1 = pd.merge(left = df1.copy(), right = df2.copy(), left_on = 'TeamTeamID', how = 'left', right_index = True)
    m2 = pd.merge(left = m1, right = df2.copy(), left_on = 'OppTeamID', how = 'left', right_index = True, suffixes={"_Team", "_Opp"})
    return m2.copy()

def MatchMaker(input_df):
    # Might want to add the "201X_" year string later.
    match_df = input_df.copy()
    match_df['Match'] = match_df['TeamTeamID'].astype(str) + "_" + match_df['OppTeamID'].astype(str)
    match_df.drop(['TeamTeamID', 'OppTeamID'], axis = 1, inplace = True)
    return match_df.copy()

def QuickModelScreen(input_df):
    X = train_df.copy().drop('Won', axis = 1).set_index('Match')
    y = train_df['Won'].copy()
    
    X_train, X_hold, y_train, y_hold = train_test_split(X,y, test_size = 0.25, random_state = 1)
    
    ss = StandardScaler()
    Xs_train = ss.fit_transform(X_train)
    Xs_hold = ss.transform(X_hold)
    
    model = LogisticRegressionCV(Cs = np.logspace(-3,3,7), cv = 3)
    model.fit(Xs_train, y_train)
    
    print("LogReg :  train score :", model.score(Xs_train, y_train).round(2), "| Holdout Score:", model.score(Xs_hold, y_hold).round(2),\
          "| CV score:", cross_val_score(model, Xs_hold,y_hold, cv =5).mean().round(2))
    
    model = RandomForestClassifier()
    model.fit(Xs_train, y_train)
    
    print("Tree   :  train score :", model.score(Xs_train, y_train).round(2), "| Holdout Score:", model.score(Xs_hold, y_hold).round(2),\
          "| CV score:", cross_val_score(model, Xs_hold,y_hold, cv =5).mean().round(2))
    
    model = KNeighborsClassifier()
    model.fit(Xs_train, y_train)
    
    print("KNN    :  train score :", model.score(Xs_train, y_train).round(2), "| Holdout Score:", model.score(Xs_hold, y_hold).round(2),\
          "| CV score:", cross_val_score(model, Xs_hold,y_hold, cv =5).mean().round(2))

In [402]:
regstats2017 = StatGrabber(df_reg,[2017], 'Reg')

target_df = df_NCAA.loc[df_NCAA['Season'] == 2017, ['TeamTeamID', 'OppTeamID', 'Won']].reset_index(drop = True)

train_df = SingleMerger(target_df, regstats2016)

train_df = MatchMaker(train_df)

QuickModelScreen(train_df)

LogReg :  train score : 0.76 | Holdout Score: 0.65 | CV score: 0.66
Tree   :  train score : 0.98 | Holdout Score: 0.53 | CV score: 0.56
KNN    :  train score : 0.77 | Holdout Score: 0.71 | CV score: 0.64


In [403]:
regstats2016 = StatGrabber(df_reg,[2016], 'Reg')
regstats2017 = StatGrabber(df_reg, [2017], 'NCAA')

target_df = df_NCAA.loc[df_NCAA['Season'] == 2017, ['TeamTeamID', 'OppTeamID', 'Won']].reset_index(drop = True)

train_df = SingleMerger(target_df, regstats2016).copy()
train_df = SingleMerger(train_df, NCAAstats2016)

train_df = MatchMaker(train_df).copy()

QuickModelScreen(train_df)

LogReg :  train score : 0.79 | Holdout Score: 0.65 | CV score: 0.71
Tree   :  train score : 0.98 | Holdout Score: 0.56 | CV score: 0.62
KNN    :  train score : 0.72 | Holdout Score: 0.65 | CV score: 0.69


In [404]:
regstats2013 = StatGrabber(df_reg,[2013], 'Reg')
regstats2014 = StatGrabber(df_reg,[2014], 'Reg')
regstats2015 = StatGrabber(df_reg, [2015], 'Reg')
regstats2016 = StatGrabber(df_reg,[2016], 'Reg')
regstats2017 = StatGrabber(df_reg, [2017], 'Reg')

target_df = df_NCAA.loc[df_NCAA['Season'] == 2017, ['TeamTeamID', 'OppTeamID', 'Won']].reset_index(drop = True)

train_df = SingleMerger(target_df, regstats2017).copy()
train_df = SingleMerger(train_df, regstats2017)
train_df = SingleMerger(target_df, regstats2016).copy()
train_df = SingleMerger(train_df, regstats2016)
train_df = SingleMerger(target_df, regstats2015).copy()
train_df = SingleMerger(train_df, regstats2015)
train_df = SingleMerger(target_df, regstats2014).copy()
train_df = SingleMerger(train_df, regstats2014)

train_df = MatchMaker(train_df).copy()

QuickModelScreen(train_df)

LogReg :  train score : 0.68 | Holdout Score: 0.56 | CV score: 0.53
Tree   :  train score : 0.99 | Holdout Score: 0.5 | CV score: 0.49
KNN    :  train score : 0.75 | Holdout Score: 0.62 | CV score: 0.55


In [405]:
train_df = SingleMerger(target_df, regstats2017).copy()
train_df = SingleMerger(train_df, regstats2017)
train_df = SingleMerger(target_df, regstats2016).copy()
train_df = SingleMerger(train_df, regstats2016)

train_df = MatchMaker(train_df).copy()

QuickModelScreen(train_df)

LogReg :  train score : 0.74 | Holdout Score: 0.71 | CV score: 0.68
Tree   :  train score : 0.97 | Holdout Score: 0.68 | CV score: 0.48
KNN    :  train score : 0.77 | Holdout Score: 0.71 | CV score: 0.64


In [406]:
train_df = SingleMerger(target_df, regstats2017).copy()
train_df = SingleMerger(train_df, regstats2017)

train_df = MatchMaker(train_df).copy()

QuickModelScreen(train_df)

LogReg :  train score : 0.72 | Holdout Score: 0.5 | CV score: 0.62
Tree   :  train score : 0.98 | Holdout Score: 0.5 | CV score: 0.41
KNN    :  train score : 0.68 | Holdout Score: 0.68 | CV score: 0.49


In [407]:
train_df = SingleMerger(target_df, regstats2016).copy()
train_df = SingleMerger(train_df, regstats2016)

train_df = MatchMaker(train_df).copy()

QuickModelScreen(train_df)

LogReg :  train score : 0.74 | Holdout Score: 0.71 | CV score: 0.68
Tree   :  train score : 0.98 | Holdout Score: 0.65 | CV score: 0.5
KNN    :  train score : 0.77 | Holdout Score: 0.71 | CV score: 0.64


In [ ]:
## Looks like it's best to use Reg stats from the current year, and the year before, gives the best CV scores.